<a href="https://colab.research.google.com/github/DevYuriCardoso/Automatizacao-Simples-em-Python/blob/main/C%C3%B3pia_de_RVC_Crepe_v2_(Apenas_1_passo)_27_07_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### GUIA UTILIZANDO RVC (AI COVER):
https://docs.google.com/document/d/1Mmoocy4luVSFRqqGI1NGQHbMGKMNRPNyFrhJRuLEkcs/edit?usp=sharing

### GUIA TREINANDO NO RVC
https://docs.google.com/document/d/1NrZNvjydOy2HWtPsSlR6DUiKo-xC1UPZ

### GUIA ORIGINAL POR KALOMAZE:
https://docs.google.com/document/d/13_l1bd1Osgz7qlAZn-zhklCbHpVRk6bYOuAuB78qmsE/edit

##**AVISO** Este Colab é uma versão traduzida em PT-BR do guia feito por @kalomaze do AI HUB original. Link acima.

####Discord AI HUB Brasil: https://discord.gg/aihubbrasil
####Discord AI HUB Americano: https://discord.gg/aihub

Observação: 'mangio-crepe' é superior ao 'crepe' tanto em treinamento quanto inferência. O 'hop size' não vai ser configuravel corretamente caso use o 'crepe'.

#Etapa 1: Aperte no Play e espere terminar (Cerca de 1 Minuto)


In [ ]:
#@title Initialize External Code
import time
import os
import subprocess
import shutil

# Store the current working directory
current_path = os.getcwd()

# Clear the /content/ directory
shutil.rmtree('/content/')
os.makedirs('/content/', exist_ok=True)

# Change the current working directory back to the original path
os.chdir(current_path)

start_time = time.time()

!nvidia-smi
!git clone https://github.com/kalomaze/externalcolabcode.git /content/Retrieval-based-Voice-Conversion-WebUI/utils

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for utils Download: {elapsed_time} seconds")

In [ ]:
#@title Install Dependencies and Clone Github Repository

import threading

start_time = time.time()

os.chdir("/content/Retrieval-based-Voice-Conversion-WebUI/")
from utils.dependency import *
from utils.clone_alt import *
os.chdir("..")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for imports: {elapsed_time} seconds")

#@markdown This will forcefully update dependencies even after the initial install seemed to have functioned.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown This will force temporary storage to be used, so it will download dependencies every time instead of on Drive. Not needed, unless you really want that 160mb storage. (Turned on by default for non-training colab to boost the initial launch speed)
ForceTemporaryStorage = False #@param{type:"boolean"}

# Setup environment
print("Attempting to setup environment dependencies...")
print("\n----------------------------------------")

start_time_setup = time.time()
setup_environment(ForceUpdateDependencies, ForceTemporaryStorage)
end_time_setup = time.time()
elapsed_time_setup = end_time_setup - start_time_setup
print(f"Time taken for setup environment: {elapsed_time_setup} seconds")

print("----------------------------------------\n")
print("Attempting to clone necessary files...")
print("\n----------------------------------------")

start_time_clone = time.time()
clone_repository(True)
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/

end_time_clone = time.time()
elapsed_time_clone = end_time_clone - start_time_clone
print(f"Time taken for clone repository: {elapsed_time_clone} seconds")

print("----------------------------------------\n")
print("Cell completed.")

total_time = elapsed_time + elapsed_time_setup + elapsed_time_clone
print(f"Total time taken: {total_time} seconds")

!pip install -q stftpitchshift==1.5.1

In [ ]:
# @title Tradução PT-BR
# Instalando tradução PT-BR
os.chdir('/content/')
!wget https://raw.githubusercontent.com/MrM0dZ/EasyGUI-PT-BR-AHBR/main/i18n/pt_BR.json -P /content/Retrieval-based-Voice-Conversion-WebUI/i18n/
!wget https://raw.githubusercontent.com/MrM0dZ/EasyGUI-PT-BR-AHBR/main/docs/faq_pt_BR.md -P /content/Retrieval-based-Voice-Conversion-WebUI/docs/

!rm -f /content/Retrieval-based-Voice-Conversion-WebUI/EasierGUI.py;
!wget https://raw.githubusercontent.com/MrM0dZ/EasyGUI-PT-BR-AHBR/main/EasierGUI.py -P /content/Retrieval-based-Voice-Conversion-WebUI/

!rm -f /content/Retrieval-based-Voice-Conversion-WebUI/infer-web.py
!wget https://raw.githubusercontent.com/MrM0dZ/EasyGUI-PT-BR-AHBR/main/infer-web.py -P /content/Retrieval-based-Voice-Conversion-WebUI/


#Etapa 2: Iniciar EasyGUI

Aguarde o "public URL" aparecer e clique no link.

![alt text](https://i.imgur.com/ZjuyG29.png)


In [ ]:
import os
import threading
import time
from utils import backups

#@markdown #Clique no Play para iniciar

#@markdown Deixe essa opção marcada para utilizar o EasyGUI.
easy_gui = True #@param{type:"boolean"}

#@markdown Deixe essa opção marcada para utilizar os backups automaticos.
auto_backups = True #@param{type:"boolean"}

LOGS_FOLDER = '/content/Retrieval-based-Voice-Conversion-WebUI/logs'
if not os.path.exists(LOGS_FOLDER):
    os.makedirs(LOGS_FOLDER)

WEIGHTS_FOLDER = '/content/Retrieval-based-Voice-Conversion-WebUI/weights'
if not os.path.exists(WEIGHTS_FOLDER):
    os.makedirs(WEIGHTS_FOLDER)

def extract_wav2lip_tar_files():
    !wget -P /content/ https://github.com/777gt/EVC/raw/main/wav2lip-HD.tar.gz
    !wget -P /content/ https://github.com/777gt/EVC/raw/main/wav2lip-cache.tar.gz

    with tarfile.open('/content/wav2lip-cache.tar.gz', 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join('/', member.name)
            try:
                tar.extract(member, '/')
            except:
                pass
    with tarfile.open('/content/wav2lip-HD.tar.gz') as tar:
        tar.extractall('/content')

def start_web_server():
    %cd /content/Retrieval-based-Voice-Conversion-WebUI
    %load_ext tensorboard
    %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/audios
    if easy_gui:
        !pip install -q gTTS
        !pip install -q elevenlabs
        !wget https://github.com/777gt/-EVC-/raw/main/audios/someguy.mp3 -P /content/Retrieval-based-Voice-Conversion-WebUI/audios/
        !wget https://github.com/777gt/-EVC-/raw/main/audios/somegirl.mp3 -P /content/Retrieval-based-Voice-Conversion-WebUI/audios/
        extract_wav2lip_tar_files()
        !python3 EasierGUI.py --colab --pycmd python3
    else:
        !python3 infer-web.py --colab --pycmd python3

# Import the Google Drive backup before starting the server only if auto_backups is True
if auto_backups:
    backups.import_google_drive_backup()


# Start the web server in a separate thread
web_server_thread = threading.Thread(target=start_web_server)
web_server_thread.start()

# Run the backup loop in the main thread only if auto_backups is True
if auto_backups:
    backups.backup_files()
else:
    while True:
        time.sleep(10) # sleep for 10 seconds



#Credits
##Creditos
**MrM0dZ** - Tradução desse Colab, Guia do RVC, Revisão EasyGUI<br>
**Rafa.Godoy** - Tradução do EasyGUI,RVC GUI e FAQ em PT-BR.<br>
**Krisp** - Internacionalização do Projeto
**Rejekts** - Autor original do colab author. Criou o Easy GUI para RVC<br>
**RVC-Project dev team** - Desenvolvedores Originais do software RVC<br>
**Mangio621** - Desenvolvedor do fork do RVC que adicionou suporte ao 'crepe'.<br>
**Kalomaze** - Criador do colab original em inglês. Todo o codigo aqui funcionou devido aos seus esforços.